In [9]:
import phenopackets as pp
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', None)
from pyphetools.creation import *
import logging

def done():
    logging.info("Done..")
    
from hpotk import Ontology
from hpotk.ontology.load.obographs import load_ontology

logging.info("Loading graph data..")
o: Ontology = load_ontology('http://purl.obolibrary.org/obo/hp.json')
metadata = MetaData(created_by="ORCID:0000-0002-0736-9199")
metadata.default_versions_with_hpo(version=o.version)



In [5]:
logging.info('Loading neonatal data [{0}].'.format("/Users/gargam/Develop/neonatal-packets/examples/pexample.csv"))
df = pd.read_csv("/Users/gargam/Develop/neonatal-packets/examples/pexample.csv", sep="\t", keep_default_na=False)
done()
logging.info("Identifying hpo columns..")
logging.info("Scanning samples that have no phenotypes..")
phenotype_col = "Anomalies with HPO codes (Screening)"
empty_idx = np.where(df[phenotype_col].eq('') | df[phenotype_col].isna())[0].tolist()
empty_samples = [df.iloc[i, 0] for i in empty_idx]
logging.info(empty_samples)

logging.info("Dropping empties..")
df.drop(empty_idx, axis=0, inplace=True)

In [8]:
def to_hpterm(terms, o):
    result = []
    for term in terms:
        term = o.get_term(term.replace(' ', ''))
        if term == None:
            print(terms)
        result.append(HpTerm(term.identifier.value, term.name))
    return result

In [10]:
logging.info("Mapping rows to phenopackets...")
samples = []
for index, row in df.iterrows():
    terms = to_hpterm(row[2].split(","), o)
    samples.append(Individual(row[0], terms, interpretation_list=[]))

In [11]:
Individual.output_individuals_as_phenopackets(samples, metadata = metadata.to_ga4gh())

101